<a href="https://colab.research.google.com/github/aidantze/pesta-la-vista/blob/yolo_v8/yolo_v8_aid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolo v8

Constants

In [6]:
# YOLO constants
RESOLUTION = 512
EPOCHS = 1                        # set to 10 for testing, leave as 30 default

# Noise and filter constants
APPLY_CORRUPTION = True
CORRUPT_TYPE = 'gaussian_noise'   # one of: 'gaussian_noise', 'salt_pepper_noise', 'gaussian_blur'
CORRUPT_STRENGTH = 0.05           # e.g., 0.05 = 5% noise or 5x5 kernel blur

Setup

In [7]:
!pip install -q ultralytics kagglehub pyyaml

import numpy as np
import cv2
import kagglehub
import pathlib
import yaml
import shutil
from ultralytics import YOLO
import torch
import time

# Check for T4 availability on Colab (DON'T CHANGE)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected. Go to Runtime -> Change runtime type ->  GPU")

# Kaggle Download via CLI API (see their website - DON'T CHANGE)
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

# Saved the images to a local path to increase efficiency
local_path = pathlib.Path("/content/datasets/crop-pests")
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

# YAML CONGIF (DON'T CHANGE)
data_yaml_path = local_path / "data.yaml"
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "names": [
        "ant", "bee", "beetle", "caterpillar", "earthworm", "earwig",
        "grasshopper", "moth", "slug", "snail", "wasp", "weevil"
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)


CUDA available: True
GPU: Tesla T4
Using Colab cache for faster access to the 'crop-pests-dataset' dataset.


Noise and Filter/Blur Analysis

In [8]:

# Noise and filter/blur analysis
def apply_corruption_to_folder(source_dir, destination_dir, corruption_type, strength=0.01):
    """
    Copies images from source to destination and applies a specified corruption.

    Args:
        source_dir (pathlib.Path): Directory containing original images.
        destination_dir (pathlib.Path): Target directory for corrupted images.
        corruption_type (str): 'gaussian_noise', 'salt_pepper_noise', 'gaussian_blur'.
        strength (float/int): Magnitude of the corruption.
    """
    if destination_dir.exists():
        shutil.rmtree(destination_dir)
    shutil.copytree(source_dir, destination_dir)

    if corruption_type == 'gaussian_noise':
        # Mean=0, standard deviation=strength * 255
        sigma = int(strength * 255)
        print(f"\nApplying {corruption_type} (Sigma: {sigma}) to images in {destination_dir.name}...")

        for img_file in destination_dir.glob('*.jpg'): # Adjust extension if needed
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            noise = np.random.normal(0, sigma, img.shape).astype('uint8')
            corrupted_img = cv2.add(img, noise)

            # Save the corrupted image, overwriting the copy
            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'salt_pepper_noise':
        ratio = strength # Ratio of pixels to corrupt
        print(f"\nApplying {corruption_type} (Ratio: {ratio}) to images in {destination_dir.name}...")

        for img_file in destination_dir.glob('*.jpg'): # Adjust extension if needed
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            corrupted_img = img.copy()
            total_pixels = img.size
            num_salt_pepper = int(ratio * total_pixels / img.shape[2]) # Total pixels / num channels

            # Salt noise (white)
            coords = [np.random.randint(0, i - 1, num_salt_pepper) for i in img.shape]
            corrupted_img[coords[0], coords[1], coords[2]] = 255

            # Pepper noise (black)
            coords = [np.random.randint(0, i - 1, num_salt_pepper) for i in img.shape]
            corrupted_img[coords[0], coords[1], coords[2]] = 0

            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'gaussian_blur':
        # Kernel size = strength (must be odd), sigma=0 (auto)
        ksize = int(strength * 100) if strength % 2 != 0 else int(strength * 100) + 1
        print(f"\nApplying {corruption_type} (ksize: {ksize}) to images in {destination_dir.name}...")

        for img_file in destination_dir.glob('*.jpg'): # Adjust extension if needed
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            corrupted_img = cv2.GaussianBlur(img, (ksize, ksize), 0)

            cv2.imwrite(str(img_file), corrupted_img)

    print("Corruption application complete.")


if (APPLY_CORRUPTION):
  # Create a new validation directory for the corrupted test set
  original_val_path = local_path / "valid" / "images"
  corrupt_val_path = local_path / "valid_noisy" / "images"

  # Apply corruption to the copied test set
  # We use 'valid' here since YOLOv8 validation defaults to the 'val' split name.
  apply_corruption_to_folder(
      original_val_path,
      corrupt_val_path,
      CORRUPT_TYPE,
      CORRUPT_STRENGTH
  )

  # Update the YAML to point to the corrupted validation set for the experiment
  data_cfg_corrupted = data_cfg.copy()
  data_cfg_corrupted['val'] = "valid_noisy/images"
  data_yaml_path_corrupted = local_path / "data_corrupted.yaml"

  with open(data_yaml_path_corrupted, "w") as f:
      yaml.safe_dump(data_cfg_corrupted, f)
  print("Wrote corrupted config:", data_yaml_path_corrupted)


Applying gaussian_noise (Sigma: 12) to images in images...
Corruption application complete.
Wrote corrupted config: /content/datasets/crop-pests/data_corrupted.yaml


Train model and prediction

In [ ]:
# Training the Model (as per YOLOv8n website)
model = YOLO("yolov8n.pt")

# See changes made to increase for Colab
start = time.time()
train_res = model.train(
    data=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=RESOLUTION,
    batch=-1,            # auto batch size based on GPU memory
    device=0,            # use GPU (CUDA:0)
    workers=4,           # increase data loading threads
    cache=True,          # cache dataset in RAM/Disk for speed
    amp=True,            # mixed precision
    freeze=10,           # aka for transfer learning (recommended by AI)
    mosaic=0.5,          # light augmentations
    mixup=0.0,
    copy_paste=0.0,
    project="pests_fast",
    name="yolov8n_colab",
    plots=False
)
end = time.time()
train_time = end - start

# Validate the Model (as per YOLOv8n website)
start = time.time()
val_results = model.val(
    data=str(data_yaml_path),
    split="test",
    batch=16,
    device=0,
    workers=2
)
end = time.time()
val_time = end - start
print(val_results)

# Test the Model (as per YOLOv8n website)
sample_dir = local_path / "test" / "images"
start = time.time()
test_results = model.predict(
    source=str(sample_dir),
    batch=16,
    imgsz=RESOLUTION,
    device=0,
    save=True,
    project="runs/detect",
    name="pest_predictions"
)
end = time.time()
test_time = end - start

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/crop-pests/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=yolov8n_colab11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

Output Metrics

In [ ]:
# Mean Average Precision (mAP)
mAP50 = val_results.results_dict['metrics/mAP50(B)']
mAP50_95 = val_results.results_dict['metrics/mAP50-95(B)']

# Precision, Recall, and F1-score
precision = val_results.results_dict['metrics/precision(B)']
recall = val_results.results_dict['metrics/recall(B)']
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Note: YOLO does not directly output a single 'Accuracy' metric in the classification sense,
# nor a single 'Area Under the Curve (AUC)' value; mAP is the primary AUC equivalent.

print(f"Results for YOLO with {f"{CORRUPT_TYPE}, strength set to {CORRUPT_STRENGTH}" if APPLY_CORRUPTION else "no filters"}:\n")
print(f"Mean Average Precision (mAP@0.50): {mAP50:.4f}")
print(f"Mean Average Precision (mAP@0.50-0.95): {mAP50_95:.4f}")
print(f"Precision (Box): {precision:.4f}")
print(f"Recall (Box): {recall:.4f}")
print(f"F1-Score (Derived): {f1_score:.4f}\n")

def format_time(seconds):
    """Converts total seconds into minutes and format."""
    mins, secs = divmod(seconds, 60)
    return f"{int(mins):0d}m {secs:.2f}s"

print(f"Training Time (Total): {format_time(train_time)}")
print(f"Testing/Validation Time (Total): {format_time(test_time)}")